# Scenario

I work for an online reatailer that sells a wide variety of different items to consumers. Each sale through the online storefront is recorded in a database with various characteristics, including invoice number, product stock code, quantity purchased, sale price, description, etc. As part of my data science business project, my team has been tasked with creating models that can give the business more insights into its customers' behaviour, and even predict future purchasing decisions.

Before I can do that, however, I need to go through the priliminary ETL process to make sure the data is in an acceptable state.


# Import software libraries

In [ ]:
# Import required libraries.
import sys           # Read system parameters.
import pandas as pd  # Manipulate and analyze data.
import sqlite3       # Manage SQL databases.

# Summarize software libraries used.
print('Libraries used in this project:')
print('- Python {}'.format(sys.version))
print('- pandas {}'.format(pd.__version__))
print('- sqlite3 {}'.format(sqlite3.sqlite_version))

Libraries used in this project:
- Python 3.7.6 | packaged by conda-forge | (default, Mar 23 2020, 23:03:20) 
[GCC 7.3.0]
- pandas 1.1.3
- sqlite3 3.30.1


# Examine the database

In [ ]:
# Connect to SQLite database.

conn = sqlite3.connect('data/prod_sample.db')
conn

In [ ]:
# List all the tables in the database.
def sql_fetch(conn):

    cursorObj = conn.cursor()

    cursorObj.execute('SELECT name from sqlite_master where type= "table"')

    print(cursorObj.fetchall())

sql_fetch(conn)

[('stock_description',), ('online_retail_history',)]


# Read data from the `online_retail_history` table

In [ ]:
# Select everything from the online_retail_history table.

query = 'SELECT * FROM online_retail_history'

# Use the read_sql function in pandas to read a query into a DataFrame.

retail_table = pd.read_sql(query,conn)

# Preview the first five rows of the data.

retail_table.head()

,Invoice,StockCode,Quantity,InvoiceDate,Price,CustomerID,Country,TotalAmount
0,536365,85123A,6,2010-12-01 08:26:00,2.55,u1785,United Kingdom,15.30
1,536367,84879,32,2010-12-01 08:34:00,1.69,u13047,United Kingdom,54.08
2,536373,85123A,6,2010-12-01 09:02:00,2.55,u1785,United Kingdom,15.30
3,536375,85123A,6,2010-12-01 09:32:00,2.55,u1785,United Kingdom,15.30
4,536378,20725,10,2010-12-01 09:37:00,1.65,u14688,United Kingdom,16.50


In [ ]:
# Get the shape of the data.

retail_table.shape

(15321, 8)

# Read data from the `stock_description` table

In [ ]:
# Select everything from the stock_description table.

query = 'SELECT * FROM stock_description'

# Read a query into a DataFrame

stock_table = pd.read_sql(query,conn)

# First five rows of the data.

stock_table.head()

,StockCode,Description
0,10002,INFLATABLE POLITICAL GLOBE
1,10080,GROOVY CACTUS INFLATABLE
2,10120,DOGGY RUBBER
3,10123C,HEARTS WRAPPING TAPE
4,10124A,SPOTS ON RED BOOKCOVER TAPE


In [ ]:
# Get the shape of the data.

stock_table.shape

(15321, 8)

# Aggregate the `online_retail_history` and `stock_description` datasets

In [ ]:
# Query that aggregate the two datasets so that I have the stock descriptions as well as the stock code.

query = ''' SELECT r.*, s.Description
            FROM online_retail_history r
                LEFT JOIN stock_description s
                    ON r.StockCode = s.StockCode
        '''

# Read a query into a DataFrame.

retail_agg = pd.read_sql(query, conn)

# First five rows of the data.

retail_agg.head()

,Invoice,StockCode,Quantity,InvoiceDate,Price,CustomerID,Country,TotalAmount,Description
0,536365,85123A,6,2010-12-01 08:26:00,2.55,u1785,United Kingdom,15.30,CREAM HANGING HEART T-LIGHT HOLDER
1,536367,84879,32,2010-12-01 08:34:00,1.69,u13047,United Kingdom,54.08,ASSORTED COLOUR BIRD ORNAMENT
2,536373,85123A,6,2010-12-01 09:02:00,2.55,u1785,United Kingdom,15.30,CREAM HANGING HEART T-LIGHT HOLDER
3,536375,85123A,6,2010-12-01 09:32:00,2.55,u1785,United Kingdom,15.30,CREAM HANGING HEART T-LIGHT HOLDER
4,536378,20725,10,2010-12-01 09:37:00,1.65,u14688,United Kingdom,16.50,LUNCH BAG RED RETROSPOT


In [ ]:
# Get the shape of the data.

retail_agg.shape

(17032, 9)

# Identify and fix corrupt or unusable data

In [ ]:
# Check the value counts of the "Description" field.

retail_agg.Description.value_counts()

CREAM HANGING HEART T-LIGHT HOLDER    2174
JUMBO BAG RED RETROSPOT               1960
REGENCY CAKESTAND 3 TIER              1711
?                                     1711
PARTY BUNTING                         1615
LUNCH BAG RED RETROSPOT               1421
ASSORTED COLOUR BIRD ORNAMENT         1405
POPCORN HOLDER                        1329
LUNCH BAG  BLACK SKULL.               1271
SET OF 3 CAKE TINS PANTRY DESIGN      1257
PACK OF 72 RETROSPOT CAKE CASES       1178
Name: Description, dtype: int64

In [ ]:
# Remove rows where "Description" is just a question mark (?).

retail_cleaned = \
retail_agg[retail_agg.Description != '?']


# First five rows of the data.
print(retail_cleaned.shape)
print(retail_table.shape)
retail_cleaned.head()

(15321, 9)
(15321, 8)


,Invoice,StockCode,Quantity,InvoiceDate,Price,CustomerID,Country,TotalAmount,Description
0,536365,85123A,6,2010-12-01 08:26:00,2.55,u1785,United Kingdom,15.30,CREAM HANGING HEART T-LIGHT HOLDER
1,536367,84879,32,2010-12-01 08:34:00,1.69,u13047,United Kingdom,54.08,ASSORTED COLOUR BIRD ORNAMENT
2,536373,85123A,6,2010-12-01 09:02:00,2.55,u1785,United Kingdom,15.30,CREAM HANGING HEART T-LIGHT HOLDER
3,536375,85123A,6,2010-12-01 09:32:00,2.55,u1785,United Kingdom,15.30,CREAM HANGING HEART T-LIGHT HOLDER
4,536378,20725,10,2010-12-01 09:37:00,1.65,u14688,United Kingdom,16.50,LUNCH BAG RED RETROSPOT


# Identify and remove duplicates

In [ ]:
# Identify all duplicated data.

duplicated_data = \
retail_cleaned[retail_cleaned.duplicated(keep = False)]

print('Number of rows with duplcated data: ',
     duplicated_data.shape[0])

Number of rows with duplcated data:  223


In [ ]:
# Print the duplicated data.

duplicated_data.head()

,Invoice,StockCode,Quantity,InvoiceDate,Price,CustomerID,Country,TotalAmount,Description
176,536863,20727,1,2010-12-03 11:19:00,1.65,u17967,United Kingdom,1.65,LUNCH BAG BLACK SKULL.
178,536863,20727,1,2010-12-03 11:19:00,1.65,u17967,United Kingdom,1.65,LUNCH BAG BLACK SKULL.
496,537781,84879,8,2010-12-08 12:46:00,1.69,u17341,United Kingdom,13.52,ASSORTED COLOUR BIRD ORNAMENT
497,537781,84879,8,2010-12-08 12:46:00,1.69,u17341,United Kingdom,13.52,ASSORTED COLOUR BIRD ORNAMENT
569,537955,20725,1,2010-12-09 11:28:00,1.65,u16782,United Kingdom,1.65,LUNCH BAG RED RETROSPOT


In [ ]:
# Remove the duplicated data.

retail_cleaned_final = \
retail_cleaned[~retail_cleaned.duplicated()]

# First five rows of the data.
retail_cleaned_final.head()

,Invoice,StockCode,Quantity,InvoiceDate,Price,CustomerID,Country,TotalAmount,Description
0,536365,85123A,6,2010-12-01 08:26:00,2.55,u1785,United Kingdom,15.30,CREAM HANGING HEART T-LIGHT HOLDER
1,536367,84879,32,2010-12-01 08:34:00,1.69,u13047,United Kingdom,54.08,ASSORTED COLOUR BIRD ORNAMENT
2,536373,85123A,6,2010-12-01 09:02:00,2.55,u1785,United Kingdom,15.30,CREAM HANGING HEART T-LIGHT HOLDER
3,536375,85123A,6,2010-12-01 09:32:00,2.55,u1785,United Kingdom,15.30,CREAM HANGING HEART T-LIGHT HOLDER
4,536378,20725,10,2010-12-01 09:37:00,1.65,u14688,United Kingdom,16.50,LUNCH BAG RED RETROSPOT


# Correct date formats

In [ ]:
# Get the data types for every column in the DataFrame.

retail_cleaned_final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15206 entries, 0 to 17031
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Invoice      15206 non-null  object 
 1   StockCode    15206 non-null  object 
 2   Quantity     15206 non-null  int64  
 3   InvoiceDate  15206 non-null  object 
 4   Price        15194 non-null  float64
 5   CustomerID   12435 non-null  object 
 6   Country      15206 non-null  object 
 7   TotalAmount  15194 non-null  float64
 8   Description  15206 non-null  object 
dtypes: float64(2), int64(1), object(6)
memory usage: 1.2+ MB


In [ ]:
# Convert "InvoiceDate" to a "%Y-%m-%d" datetime format.

retail_cleaned_final_2 = retail_cleaned_final.copy()  # Work with a new object.

retail_cleaned_final_2['InvoiceDate'] = \
pd.to_datetime(retail_cleaned_final_2['InvoiceDate'],
               format = '%Y-%m-%d')

In [ ]:
# Get the data types for every column in the converted DataFrame.

retail_cleaned_final_2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15206 entries, 0 to 17031
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   Invoice      15206 non-null  object        
 1   StockCode    15206 non-null  object        
 2   Quantity     15206 non-null  int64         
 3   InvoiceDate  15206 non-null  datetime64[ns]
 4   Price        15194 non-null  float64       
 5   CustomerID   12435 non-null  object        
 6   Country      15206 non-null  object        
 7   TotalAmount  15194 non-null  float64       
 8   Description  15206 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(5)
memory usage: 1.2+ MB


# Examine the table before finishing

In [ ]:
# Preview the first five rows of the data.

retail_cleaned_final_2.head()

,Invoice,StockCode,Quantity,InvoiceDate,Price,CustomerID,Country,TotalAmount,Description
0,536365,85123A,6,2010-12-01 08:26:00,2.55,u1785,United Kingdom,15.30,CREAM HANGING HEART T-LIGHT HOLDER
1,536367,84879,32,2010-12-01 08:34:00,1.69,u13047,United Kingdom,54.08,ASSORTED COLOUR BIRD ORNAMENT
2,536373,85123A,6,2010-12-01 09:02:00,2.55,u1785,United Kingdom,15.30,CREAM HANGING HEART T-LIGHT HOLDER
3,536375,85123A,6,2010-12-01 09:32:00,2.55,u1785,United Kingdom,15.30,CREAM HANGING HEART T-LIGHT HOLDER
4,536378,20725,10,2010-12-01 09:37:00,1.65,u14688,United Kingdom,16.50,LUNCH BAG RED RETROSPOT


# Load the dataset into a pickle file

In [ ]:
# Save the dataset as a pickle file named online_history_cleaned.pickle.

retail_cleaned_final_2.to_pickle('retail_cleaned_final_2.pickle')

In [ ]:
# Close any connections to the database.

conn.close()